In [1]:
import pandas as pd
import numpy as np
from requests import get
from bs4 import BeautifulSoup as bs

import src.cleaning as cleaning

In [2]:
trucks = pd.read_csv('data/truck_transfers.csv')
april = pd.read_csv('data/april_2014.csv')

In [12]:
# stations = cleaning.stations_coordinates (april)
# stations capacity
# trucks = cleaning.coordinate_columns (trucks)
# trucks = cleaning.truck_trips_coordinates (stations, trucks)
# trucks = cleaning.time_difference (trucks)

In [27]:
df_list = []
for bike_id in april.bike_id.unique().tolist():
    df = cleaning.bike_human_truck_trips(april, bike_id)
    df_list.append(df)

/Users/gerardrius/Ironhack/projects/Final/src/cleaning.py:452: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  result_df = pd.DataFrame({'last_end': last_end, 'ended_at': etime, 'next_start': next_start, 'started_at': stime})
/Users/gerardrius/Ironhack/projects/Final/src/cleaning.py:452: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  result_df = pd.DataFrame({'last_end': last_end, 'ended_at': etime, 'next_start': next_start, 'started_at': stime})
/Users/gerardrius/Ironhack/projects/Final/src/cleaning.py:452: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series

In [28]:
ALL_TRIPS = pd.concat(df_list, ignore_index = True)

In [29]:
ALL_TRIPS

,last_end,ended_at,next_start,started_at,time_difference,bike_id
0,2008.0,2014-04-01 00:09:25.000,2008.0,2014-04-01 07:41:30.000,0 days 07:32:05,21062.0
1,224.0,2014-04-01 07:48:08.000,224.0,2014-04-01 08:54:32.000,0 days 01:06:24,21062.0
2,360.0,2014-04-01 08:58:20.000,360.0,2014-04-01 15:03:40.800,0 days 06:05:20.800000,21062.0
3,306.0,2014-04-02 09:19:43.200,306.0,2014-04-02 15:25:04.000,0 days 06:05:20.800000,21062.0
4,147.0,2014-04-02 15:32:24.000,147.0,2014-04-02 17:22:38.000,0 days 01:50:14,21062.0
...,...,...,...,...,...,...
741298,305.0,2014-04-29 08:39:25.000,305.0,2014-04-29 09:01:35.000,0 days 00:22:10,19206.0
741299,430.0,2014-04-29 18:42:23.000,430.0,2014-04-29 19:00:09.000,0 days 00:17:46,19025.0
741300,242.0,2014-04-29 19:05:58.000,242.0,2014-04-29 20:01:48.000,0 days 00:55:50,19025.0
741301,324.0,2014-04-29 20:09:57.000,324.0,2014-04-29 20:32:11.000,0 days 00:22:14,19025.0
